# Model Entailment 1

This is the first of several notebooks to explore logical entailments in SysML v2 models, as well as the means by which to encode them.

In [ ]:
import json
import pymbe.api as pm

from pathlib import Path

from typing import Any, Collection, Dict, List, Tuple, Union

from pymbe.model import Model, Element
from pymbe.model_modification import build_from_classifier_pattern, \
                                    new_element_ownership_pattern, \
                                    build_from_binary_relationship_pattern, \
                                    build_unioning_superset_classifier, \
                                    build_from_feature_pattern, \
                                    build_from_binary_connector_pattern

from pymbe.query.metamodel_navigator import is_type_undefined_mult, \
                                    is_multiplicity_one, \
                                    is_multiplicity_specific_finite, \
                                    get_finite_multiplicity_types, \
                                    identify_connectors_one_side, \
                                    get_lower_multiplicty, \
                                    get_upper_multiplicty

from uuid import uuid4

## Load up and explore basic model

Load up a basic model in order to have basic package and namespace into which to add additional elements.

In [ ]:
filename = "Model_Loader_Test_Level_2"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()).parent.parent / "tests/fixtures" / filename

level2 = pm.Model.load_from_post_file(json_file)
level2

In [ ]:
len(level2.elements)

In [ ]:
classifiers = [ele for ele in level2.elements.values() if ele._metatype == 'Classifier']
classifiers

Locate the root package of the model to which new elements will be added.

In [ ]:
base_package = classifiers[0].owningRelationship.owningRelatedElement
base_package

## Example Application of Reasoning 1 - Build Common Class from Examples

In this example, we look at a series of specific examples of an item, 4 bicycle wheels.

First, add these elements to the model.

In [ ]:
bw1 = build_from_classifier_pattern(owner=base_package, name="Bike Wheel #1", model=level2, specific_fields={}, metatype="Classifier")
bw2 = build_from_classifier_pattern(owner=base_package, name="Bike Wheel #2", model=level2, specific_fields={}, metatype="Classifier")
bw3 = build_from_classifier_pattern(owner=base_package, name="Bike Wheel #3", model=level2, specific_fields={}, metatype="Classifier")
bw4 = build_from_classifier_pattern(owner=base_package, name="Bike Wheel #4", model=level2, specific_fields={}, metatype="Classifier")

In [ ]:
bw1.owningRelationship

Now, create a new superclass that all of these specific wheels can be grouped into.

In [ ]:
new_wheel = build_unioning_superset_classifier(classes=[bw1, bw2, bw3, bw4],
                                    super_name="Bike Wheel",
                                    model=level2,
                                    owner=base_package,
                                    added_fields={})

Check that the wheel has its specific versions.

In [ ]:
new_wheel.reverseSubclassification

In [ ]:
base_package.ownedMember

## Developing unrolling rule around connectors

Rule 1 - find connectors with ends that have a multiplicity of 1 and then specialize them.

In [ ]:
connectors = [ele for ele in level2.elements.values() if ele._metatype == 'Connector']

In [ ]:
efms = [ele for ele in level2.elements.values() if ele._metatype == 'EndFeatureMembership']
efms

In [ ]:
for end_feature in connectors[0].throughEndFeatureMembership:
    print(end_feature)
    if 'throughReferenceSubsetting' in end_feature._derived:
        print(f"Feature references {end_feature.throughReferenceSubsetting[0]}")

In [ ]:
features = [ele for ele in level2.elements.values() if ele._metatype == 'Feature']
features

Look for multiplicity ranges in the model that are set to 1.

In [ ]:
[is_type_undefined_mult(ft) for ft in features]

In [ ]:
[is_multiplicity_one(ft) for ft in features]

In [ ]:
[is_multiplicity_specific_finite(ft) for ft in features]

Find all types (classifiers and features) that have a declared multiplicity that is finite.

In [ ]:
finite_mult = get_finite_multiplicity_types(level2)
finite_mult

In [ ]:
get_lower_multiplicty(finite_mult[0])

In [ ]:
get_upper_multiplicty(finite_mult[0])

Find where ends the connection is bound to other features in the model.

In [ ]:
refsubs = [ele for ele in level2.elements.values() if ele._metatype == 'ReferenceSubsetting']
refsubs

In [ ]:
conns_to_cover = identify_connectors_one_side(connectors)
conns_to_cover

In [ ]:
conns_to_cover[0].source

The function below shows the creation of a connector which requires many elements to be created (the connection itself, the ends, references out to other Features in the mind, specialization relationship.

In [ ]:
test_conn = build_from_binary_connector_pattern(
    name="Test Connector",
    source_role_name="source end",
    target_role_name="target end",
    source=bw1,
    target=bw2,
    model=level2,
    metatype="Connector",
    owner=base_package,
    specific_fields={}
)

In [ ]:
test_conn._data

Get the root package again for the new connections to be owned by.

In [ ]:
top_elements = [ele for ele in level2.ownedElement if ele._metatype == "Namespace"][0].throughOwningMembership
top_package = [ele for ele in top_elements if ele._metatype == "Package"][0]
top_package

In [ ]:
connectors = [ele for ele in level2.elements.values() if ele._metatype == 'Connector']
connectors

Look at end feature memberships and the links between the classifiers connected.

In [ ]:
connectors[1].ownedRelationship

In [ ]:
connectors[1].ownedRelationship[0].target[0].throughReferenceSubsetting

In [ ]:
connectors[1].ownedRelationship[1].target[0].throughReferenceSubsetting